In [1]:
# calc_eff.py — port of calc_eff.m using tau()
import time, math
import numpy as np
import matplotlib.pyplot as plt
from tau import tau

# fzero(@tau,[0,1],...) 대체
def fzero_like(func, a, b, args=(), max_iter=100, tol=1e-12, subdiv=1000):
    fa = func(a, *args)
    fb = func(b, *args)
    if fa == 0.0:
        return a
    if fb == 0.0:
        return b
    xs = np.linspace(a, b, subdiv+1)
    fs = np.array([func(x, *args) for x in xs])
    signs = np.sign(fs)
    for i in range(len(xs)-1):
        if signs[i] == 0:
            return xs[i]
        if signs[i]*signs[i+1] < 0:
            a_i, b_i = xs[i], xs[i+1]
            fa_i, fb_i = fs[i], fs[i+1]
            for _ in range(max_iter):
                m = 0.5*(a_i+b_i)
                fm = func(m, *args)
                if abs(fm) < tol or abs(b_i-a_i) < tol:
                    return m
                if fa_i*fm <= 0:
                    b_i, fb_i = m, fm
                else:
                    a_i, fa_i = m, fm
            return 0.5*(a_i+b_i)
    idx = int(np.argmin(np.abs(fs)))
    return float(xs[idx])

if __name__ == '__main__':
    # MATLAB params
    n = np.arange(1, 200+1)  # 1:200
    W0 = 31
    m = 5
    M = 8
    L_MPDU = 2000.0
    Data_Rate = 6.6667e6

    L_PHY = 40e-6
    L_Trigger = 100e-6
    L_BACK = 68e-6
    SIFS = 16e-6

    T = 8*L_MPDU/Data_Rate + 3*SIFS + 3*L_PHY + L_BACK

    tvals = np.zeros(len(n))
    pvals = np.zeros(len(n))
    eff   = np.zeros(len(n))
    Th    = np.zeros(len(n))
    Ds    = np.zeros(len(n))
    D     = np.zeros(len(n))

    t0 = time.time()

    for i, ni in enumerate(n):
        tval = fzero_like(lambda x, ni, W0, m, M: tau(x, ni, W0, m, M), 0.0, 1.0, args=(ni, W0, m, M))
        tvals[i] = tval
        pi = 1.0 - (1.0 - tval/M)**(ni - 1)
        pvals[i] = pi
        Ps  = tval*(1.0 - pi)
        Es  = ni*Ps
        eff[i] = Es/M

        Ptr = 1.0 - (1.0 - tval/M)**ni
        denom = 1.0 - (1.0 - (tval/M))**ni
        if denom == 0.0:
            Ps_ = 0.0
        else:
            Ps_ = ( ni*(tval/M)*(1.0 - (tval/M))**(ni - 1) ) / denom

        Th[i] = M*(Ptr*Ps_*8.0*L_MPDU) / ( (1.0-Ptr)*T + Ptr*Ps_*T + Ptr*(1.0-Ps_)*T )
        D[i]  = 1.0 / ( tval*(1.0 - (tval/M))**(ni - 1) ) if tval>0 else np.inf
        Ds[i] = 1.0 / ( 1.0 - (1.0 - tval*(1.0 - pi))**ni )

    # match MATLAB outputs
    tvals = tvals.reshape(-1,1)
    pvals = pvals.reshape(-1,1)
    eff   = eff.reshape(-1,1)
    ThMb  = (Th*1e-6).reshape(-1,1)
    Ds = Ds.reshape(-1,1)
    D  = D.reshape(-1,1)
    R  = np.hstack([n.reshape(-1,1), tvals, pvals, eff, ThMb, Ds, D])

    # Plots
    plt.figure(); plt.plot(n, eff); plt.grid(True)
    plt.xlabel('number of stations'); plt.ylabel('system efficiency')

    plt.figure(); plt.plot(n, Th*1e-6); plt.grid(True)
    plt.xlabel('number of stations'); plt.ylabel('Throughput(Mb/s)')

    elapsed = time.time() - t0
    print(f'Elapsed time: {elapsed:.2f} sec')
    plt.show()


ModuleNotFoundError: No module named 'tau'